In [67]:
import numpy as np
import sys
if "../" not in sys.path:
  sys.path.append("../") 
from lib.envs.gridworld import GridworldEnv

In [68]:
env = GridworldEnv()

### Implementation

Steps:
* Get avg reward R right
* Get avg prob P right
* Bellman backups

In [79]:
def policy_eval(policy, env, discount_factor=1.0, theta=0.00001):
    """
    Evaluate a policy given an environment and a full description of the environment's dynamics.
    
    Args:
        policy: [S, A] shaped matrix representing the policy.
        env: OpenAI env. env.P represents the transition probabilities of the environment.
            env.P[s][a] is a (prob, next_state, reward, done) tuple.
        theta: We stop evaluation once our value function change is less than theta for all states.
        discount_factor: gamma discount factor.
    
    Returns:
        Vector of length env.nS representing the value function.
    """
    # Start with a random (all 0) value function
    V = np.zeros(env.nS)
    counter = 0
    while True:
        # TODO: Implement!
        # Step 1: get the reward R
        R = np.zeros(env.nS)
        for s in range(env.nS):
            R_s = np.zeros(env.nA)
            R_s = np.array([env.P[s][a][0][2] for a in range(env.nA)])
            R[s] = np.dot(policy[s], R_s)
        # Step 2: get the transition matrix P
        P = np.zeros([env.nS, env.nS])
        for s in range(env.nS):                        
            for a in range(env.nA):
                P_s_a = np.zeros(env.nS)
                P_s_a[env.P[s][a][0][1]] = 1
                P[s,:] += policy[s][a] * P_s_a
        # Step 3: Bellman backups
        V_new = R + discount_factor*np.dot(P, V) 
        diff = np.max(np.abs(V_new - V))
        if diff < theta:
            print("Converged after %d steps." %counter)
            break
        else:
            V = V_new
            counter += 1
            if counter % 50 == 0:
                print("Iterated %d steps so far." %counter)        
    return np.array(V)

In [80]:
random_policy = np.ones([env.nS, env.nA]) / env.nA
v = policy_eval(random_policy, env)

Iterated 50 steps so far.
Iterated 100 steps so far.
Iterated 150 steps so far.
Iterated 200 steps so far.
Converged after 214 steps.


In [82]:
# Test: Make sure the evaluated policy is what we expected
expected_v = np.array([0, -14, -20, -22, -14, -18, -20, -20, -20, -20, -18, -14, -22, -20, -14, 0])
np.testing.assert_array_almost_equal(v, expected_v, decimal=2)

__Success!__

In [83]:
v

array([  0.        , -13.99988715, -19.99983277, -21.99981286,
       -13.99988715, -17.99985268, -19.99983389, -19.99983277,
       -19.99983277, -19.99983389, -17.99985268, -13.99988715,
       -21.99981286, -19.99983277, -13.99988715,   0.        ])

In [84]:
expected_v

array([  0, -14, -20, -22, -14, -18, -20, -20, -20, -20, -18, -14, -22,
       -20, -14,   0])